In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from experiments.extract import extract
from experiments.experiment import Experiment

import pandas as pd
import altair as alt
from lightgbm import LGBMRegressor

from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.preprocessing import KBinsDiscretizer, OneHotEncoder
from category_encoders import TargetEncoder, QuantileEncoder

pd.options.display.float_format = '{:,.2f}'.format

# Data Sample

In [3]:
X = extract('RENTAL')
X['month'] = pd.to_datetime(X.created_date.dt.strftime('%Y-%m-01'))
print(X.shape)
X.sample(5).T

(32846, 14)


title,Cobertura Duplex na beira da Lagoa da Tijuca e vista para a Pedra da Gávea,Rio de Janeiro - Apartamento Padrão - Barra da Tijuca,Recreio dos Bandeirantes | Casa 4 quartos,Apartamento - Rua Clóvis Beviláqua - Aluguel - Tijuca,RIO DE JANEIRO - Apartamento Padrão - RECREIO DOS BANDEIRANTES
url,/imovel/venda-cobertura-2-quartos-com-terraco-barra-da-tijuca-zona-oeste-rio-de-janeiro-rj-90m2-id-2601106922/,/imovel/aluguel-apartamento-3-quartos-barra-da-tijuca-zona-oeste-rio-de-janeiro-rj-300m2-id-2593785179/,/imovel/casa-4-quartos-recreio-dos-bandeirantes-zona-oeste-rio-de-janeiro-com-garagem-350m2-aluguel-RS24000-id-2620224730/,/imovel/apartamento-2-quartos-tijuca-zona-norte-rio-de-janeiro-com-garagem-107m2-aluguel-RS2400-id-2631308487/,/imovel/apartamento-3-quartos-recreio-dos-bandeirantes-zona-oeste-rio-de-janeiro-com-garagem-112m2-aluguel-RS3200-id-2621434561/
origin,zapimoveis,zapimoveis,vivareal,vivareal,vivareal
neighborhood,Barra da Tijuca,Barra da Tijuca,Recreio Dos Bandeirantes,Tijuca,Recreio Dos Bandeirantes
usable_area,90,300,350,107,112
unit_types,APARTMENT,APARTMENT,HOME,APARTMENT,APARTMENT
floors,3,0,1,0,0
bedrooms,2,3,4,2,3
bathrooms,2,3,3,2,3
suites,2,3,0,1,1
parking_spaces,0,2,1,1,2


In [4]:
X.groupby('neighborhood').total.describe(percentiles=[.05, .10, .25, .5, .75, .9, .95]).sort_values('count', ascending=False).head(10)

,count,mean,std,min,5%,10%,25%,50%,75%,90%,95%,max
neighborhood,,,,,,,,,,,,
Tijuca,"7,426.00","3,568.35","8,176.26",300.00,"1,700.00","1,963.00","2,430.00","3,050.50","3,987.00","5,250.00","6,304.00","684,880.00"
Barra da Tijuca,"6,905.00","19,145.27","48,331.31",150.00,"3,573.00","4,322.40","6,473.00","12,886.00","25,000.00","42,000.00","52,500.00","2,660,000.00"
Recreio Dos Bandeirantes,"4,753.00","8,425.81","7,250.34",199.00,"2,300.00","3,250.00","4,000.00","5,823.00","10,800.00","17,000.00","22,000.00","137,320.00"
Centro,"2,471.00","2,220.06","2,105.61",100.00,970.00,"1,060.00","1,330.00","1,770.00","2,380.00","3,350.00","4,181.00","28,000.00"
Copacabana,"2,198.00","6,980.32","6,797.77",130.00,"1,744.25","2,148.80","3,300.00","5,283.50","8,375.00","13,900.00","18,000.00","95,000.00"
Taquara,"2,080.00","2,193.41","2,055.99",450.00,700.00,970.00,"1,300.00","1,650.00","2,150.75","3,600.00","5,500.00","25,000.00"
Praça Seca,"1,172.00","1,634.50",737.37,500.00,900.00,990.00,"1,200.00","1,516.00","1,841.00","2,154.00","2,983.00","10,000.00"
Icaraí,"1,154.00","4,613.91","3,486.07","1,445.00","1,945.00","2,238.00","2,950.00","3,783.50","4,812.25","7,500.00","10,340.00","36,000.00"
Flamengo,"1,085.00","11,122.65","85,755.66",500.00,"1,657.00","1,945.20","2,862.00","4,727.00","10,740.00","17,772.00","21,213.60","2,000,000.00"


In [5]:
aux = X.query('lat != 0').groupby('neighborhood')[['lat', 'lon', 'total']].median().reset_index()

alt.Chart(aux).mark_point().encode(
    x=alt.X('lon', scale=alt.Scale(zero=False),),
    y=alt.Y('lat', scale=alt.Scale(zero=False)),
    size='total', tooltip=['neighborhood', 'total']
).properties(width=900, height=600).interactive()

alt.Chart(...)

In [6]:
aux = X.groupby(['month', 'neighborhood']).total.agg(['count', 'median']).reset_index()
(
    alt.Chart(aux.query('count > 10')).mark_point().encode(x='month', y='median', color='neighborhood', tooltip=['month', 'neighborhood', 'count', 'median']) +
    alt.Chart(aux.query('count > 10')).mark_line().encode(x='month', y='median', color='neighborhood')
).properties(width=900, height=600).interactive()

alt.LayerChart(...)

In [7]:
aux = X.groupby(['month', 'neighborhood']).size().reset_index(name='qtd')
alt.Chart(aux).mark_line().encode(x='month', y='qtd', color='neighborhood', tooltip=['month', 'neighborhood']).properties(width=900, height=600).interactive()

alt.Chart(...)

lat   -22.90
lon   -43.35
dtype: float64

In [26]:
aux = X.query('neighborhood == "Praça Seca" and lat != 0').reset_index()

aux['lat'] = aux['lat'] - aux['lat'].median()
aux['lon'] = aux['lon'] - aux['lon'].median()

alt.Chart(aux).mark_point().encode(
    x=alt.X('lon', scale=alt.Scale(zero=True),),
    y=alt.Y('lat', scale=alt.Scale(zero=True))
).properties(width=900, height=600).interactive()

alt.Chart(...)

In [32]:
import numpy as np

def cart_to_pol(x, y, x_c = 0, y_c = 0, deg = True):
    complex_format = x - x_c + 1j * (y - y_c)
    return np.abs(complex_format), np.angle(complex_format, deg = deg)

print(cart_to_pol(0.14, 0.10))

(0.17204650534085256, 35.53767779197438)
